# Ml-Pipes

The objective of this project is to present a gentle introduction of how a machine learning model can be trained and deployed. And how [MLFlow](https://www.mlflow.org/), [FastAPI](https://fastapi.tiangolo.com/) and [Docker](https://docs.docker.com/) can facilite a couple aspects of such a task. Within this context, this notebook represents the development part of the model, where a data scientist would create a few models, evaluate it using some metrics and select the best one based in a metric. There are a lot of different machine learning tools that can be used in order to create models. It is important to note that the objective of this notebook is to give a broad overview of and end to end machine learning process, and therefore it is recommended to have the documentation of the frameworks used here as a companion. Also, the README file from the project explains how to reproduce the whole project.

The rest of the notebook is organized as follows
- 1) The Problem;
- 2) Setting up an MLFlow Experiment;
- 3) Training different Models;
- 4) Setting the Best Model to Production.

The code bellow imports everything that will be used throughout the notebook.

In [1]:
!which python

/media/vinicius/Dados/poetry/virtualenvs/ml-pipes-VBbH4xSK-py3.8/bin/python


In [2]:
# Import Libs
import pandas as pd
pd.set_option('display.max_columns', 500)

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from settings import EXPERIMENT_NAME, TRACKING_URI, FOLDS, CREDIT_CARD_MODEL_NAME,\
     CHAMPION_METRIC, THRESHOLD  # pylint: disable=import-error
from dao.CreditCardDefault \
    import load_creditcard_dataset  # pylint: disable=import-error

from trainers.h2o_automl import H2OClassifier  # pylint: disable=import-error
from trainers.pycaret import PycaretClassifier  # pylint: disable=import-error
from trainers.spark import SparkClassifier


## 1) The problem

The first thing we need to have to build a model is a problem to solve. Here it is used as example the [Credit Card Default from Kagle](https://www.kaggle.com/mlg-ulb/creditcardfraud), where basically the objective if to predict based on a few features whether or not a client will default on its credit card. The taret variable can assume the values 1, for default, and 0 for non default. Therefore it is a binary classification problem.

Bellow the dataset is imported and the first rows of the dataset. Note that the Time column has been removed from the original dataset.

In [3]:
dataset = load_creditcard_dataset()
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
3715,1.070801,0.041820,0.667713,1.023570,-0.486857,-0.196153,-0.210674,0.215182,0.065441,0.126958,1.511280,0.263984,-1.714759,0.866891,0.793691,0.217373,-0.305565,-0.162754,-0.425243,-0.219256,-0.189233,-0.700006,0.187243,0.119866,0.091604,-0.690417,0.027305,0.022130,29.00,0
266761,1.789893,-0.411545,-1.194220,1.133385,0.986038,2.132864,-0.388684,0.575251,0.872327,0.010155,0.245467,1.437001,-0.220403,-0.025814,-1.602121,-1.290633,0.463746,-1.490560,-0.218711,-0.279185,-0.239273,-0.313765,0.231111,-0.975396,-0.082892,-0.740974,0.074463,-0.059157,35.00,0
91281,1.361927,-0.765832,0.516151,-1.002953,-0.843745,0.284293,-1.061179,0.102903,-0.703172,0.742586,0.475337,0.101678,1.247433,-0.476955,0.413005,1.980641,-0.663894,-0.072313,0.937840,0.246061,0.382232,1.007709,-0.292951,-0.771063,0.570010,0.000221,0.032445,0.010256,39.00,0
206372,-0.675469,-0.064167,0.472386,-0.820784,1.879739,-0.707702,2.234005,-1.606913,0.783636,0.823222,-0.433997,-0.183646,0.733927,-1.003084,0.458759,-0.608013,-1.325490,-0.095072,-0.195062,-0.255915,0.018727,1.407517,0.055270,0.730877,-0.400003,-0.437959,-0.987102,-0.878809,46.10,0
176518,1.867633,0.067490,-2.083760,2.949168,2.798819,4.513581,-0.459531,1.013802,-0.603922,1.316204,-0.992806,-0.000587,-0.472261,0.040007,-1.970184,-0.066826,-0.272940,-0.846602,-1.212787,-0.229559,0.064481,0.346401,0.037249,0.720629,0.339727,0.209881,-0.010506,-0.061317,19.32,0


## 2) Setting up an MLFlow Experiment

Now that a problem has been stated and some data to help solving the problem has been gathered, the next step is to setup a MLFlow experiment to log our models. **MLFlow is built upon the concept of experiments. A experiment is a series of fits, where parameters, metrics, models and artifacts can be associated with the respective fit (in an machine learning package agnostic way).**

The code bellow tries to create an experiment, if that experiments already existis then it sets the experiment to the active one.
 

In [4]:
TRACKING_URI

'http://127.0.0.1:5000'

In [5]:
EXPERIMENT_NAME='teste'
# TRACKING_URI='http://127.0.0.1:5000'

In [6]:
# mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_tracking_uri(TRACKING_URI)
try:
    experiment = mlflow.create_experiment(EXPERIMENT_NAME)
except Exception:
    client = MlflowClient()
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

mlflow.set_experiment(EXPERIMENT_NAME)

## 3) Training different Models

The ext step if to train, evaluate and log a few different models. In order to demonstrate that MLFlow allows us to use different machine learning packages we will train an H2O autoML and SkLearn models (using pycaret). Now is the time where MLFlow is put into action: For each model that if fitted it will be logged a few parameters, metrics, artifacts and the models it self. To understand how this is done it checkout the classifiers definitions in `src/trainers/` folder and the [MLFlow Logging Documentaion](https://www.mlflow.org/docs/latest/tracking.html#logging-data-to-runs), ot all happens inside the `mlflow.start_run()` context manager. 

The next cells will train different classifiers. Once they finish running you can deploy the [MLFlow Tracking UI](https://www.mlflow.org/docs/latest/tracking.html#tracking-ui) by executing `mlflow ui -p 5000 --backend-store-uri sqlite:///mlruns.db` in the terminal inside the `src/` folder. and see the results at [127.0.0.1:5000](127.0.0.1:5000).

In [7]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://127.0.0.1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio_access_key'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'MINIO_SECRET_KEY'

In [8]:
H2OClassifier(
    run_name='H2O',
    max_mem_size='3G',
    threshold=THRESHOLD,
    df=dataset,
    target_col='Class',
    sort_metric='aucpr',
    max_models=8,
    max_runtime_secs=10,
    nfolds=FOLDS,
    seed=90
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /media/vinicius/Dados/poetry/virtualenvs/ml-pipes-VBbH4xSK-py3.8/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppblhxtl4
  JVM stdout: /tmp/tmppblhxtl4/h2o_vinicius_started_from_python.out
  JVM stderr: /tmp/tmppblhxtl4/h2o_vinicius_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_vinicius_9z4i7l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
19:00:47.265: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%
Could not find exact threshold 0.5; using closest threshold found 0.4551657569282098.
Could not find exact threshold 0.5; using closest threshold found 0.4551657569282098.
Could not find exact threshold 0.5; using closest threshold found 0.4551657569282098.
Could not find exact threshold 0.5; using closest threshold found 0.4551657569282098.
Could not find exact threshold 0.5; using closest threshold found 0.4551657569282098.


In [8]:
PycaretClassifier(
        experiment_name=EXPERIMENT_NAME,
        run_name='Pycaret2',
        sort_metric='precision',
        df=dataset,
        target='Class',
        threshold=THRESHOLD,
        n_best_models=3,
        data_split_stratify=True,
        nfolds=FOLDS,
        normalize=True,
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        session_id=54321
)

,Parameters
bootstrap,False
ccp_alpha,0.0
class_weight,None
criterion,gini
max_depth,None
max_features,auto
max_leaf_nodes,None
max_samples,None
min_impurity_decrease,0.0
min_impurity_split,None


Error in logging parameter for                                 pycaret_precision_2
[Errno 2] No such file or directory: 'Hyperparameters.png'


In [6]:
SparkClassifier(
    df = dataset,
    target_col = 'Class',
    run_name = 'spark_classifier',
    max_mem_size = 4,
    n_cores = 4,
    seed = 90
)

## 4) Setting the Best Model to Production

The final step in this notebook if to set to production the model with the best selected metric, imported as `CHAMPION_METRIC`. This is done to show is is possible to create an automated workflow using MLFlow to deplot a model. However it is also possible to deplot the model using the [UI server](https://www.mlflow.org/docs/latest/model-registry.html#ui-workflow).

Once this is done you can return to the README file to check how the model is now deployed.

In [10]:
# Getting The best Model according to CHAMPION_METRIC
champion = MlflowClient().search_runs(
    experiment_ids=[
        str(
            mlflow.get_experiment_by_name(name=EXPERIMENT_NAME).experiment_id
        )
    ],
    run_view_type=ViewType.ALL,
    order_by=[f"metrics.{CHAMPION_METRIC} DESC"],
    max_results=1
)
run_id = champion[0].info.run_id

# Registering Model in model registery
model = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=CREDIT_CARD_MODEL_NAME
)

# Setting version 1
MlflowClient().update_model_version(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    description='Deploying model with model registery'
)

# Setting it to production
MlflowClient().transition_model_version_stage(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    stage="Staging"
)

Registered model 'CreditCardDefault' already exists. Creating a new version of this model...
2021/05/19 09:02:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CreditCardDefault, version 3
Created version '3' of model 'CreditCardDefault'.


<ModelVersion: creation_timestamp=1621425758707, current_stage='Staging', description='Deploying model with model registery', last_updated_timestamp=1621425758825, name='CreditCardDefault', run_id='c325b227a2d24a3994ca8e75b0201117', run_link='', source='/media/vinicius/Dados/projects/ml-pipes/mlflow_artifact_store/1/c325b227a2d24a3994ca8e75b0201117/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>